### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle
import time

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

In [3]:
print(type(Time_matrix))
print(Time_matrix.max())
print(Time_matrix.min())
print(Time_matrix.mean())
print(Time_matrix.var())

<class 'numpy.ndarray'>
11.0
0.0
3.0542857142857143
7.93705306122449


In [4]:
States_track = collections.defaultdict(dict)
Q_dict = collections.defaultdict(dict)

In [5]:
def Q_state(state):
    return '-'.join(str(e) for e in state)

In [6]:
# Initialise states to be tracked
def initialise_tracking_states():
    sample_q_values = [([1,2,3],(1,3)),([3,5,6],(4,2)),([4,10,2],(3,4)), ([2,7,0],(0,4))]    #select any 4 Q-values
    for q_values in sample_q_values:
        state = Q_state(q_values[0])
        action = q_values[1]
        States_track[state][action] = [] 

In [7]:
def save_tracking_states_old():
    """Saves the states to dictionary"""
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [8]:
initialise_tracking_states()

In [9]:
print(States_track)

defaultdict(<class 'dict'>, {'1-2-3': {(1, 3): []}, '3-5-6': {(4, 2): []}, '4-10-2': {(3, 4): []}, '2-7-0': {(0, 4): []}})


In [10]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#### Tracking the state-action pairs for checking convergence


In [11]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [12]:
num_track = 0
num_train = 0
num_track_train = 0

In [13]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = 0.95
        self.learning_rate = 0.06 # 0.06 after fix gave nothing
        self.epsilon = 1
        self.epsilon_max = 1
        #self.epsilon_decay = -0.003 #for 1k
        self.epsilon_decay = -0.0007 #for 3k
        #self.epsilon_decay = -0.0003 #for 10k
        #self.epsilon_decay = -0.0001 #for 20k
        #self.epsilon_decay = -0.00003 #for 100k
        #self.epsilon_decay = -0.000003 #for 1M
        self.epsilon_min = 0.01
        
        self.batch_size = 32      # for 24*1
        # create replay memory using deque
        self.memory = deque(maxlen=2000)
        self.states_tracked = []
        #track_input = np.zeros((1, self.state_size))
        #track_input[0] = env.state_encod_arch1([0,0,0]).reshape(1,36)
        #self.track_state = track_input[0]
        self.track_state = np.array(env.state_encod_arch1([0,0,0])).reshape(1, 36)
        #print("TRACK STATE")
        #print(self.track_state)
        self.explore = 0
        self.exploit = 0
        self.num_track = 0
        self.num_train = 0
        self.num_train_track = 0
        self.hit = False
        self.hit_index = 0

        # create main model and target model
        self.model = self.build_model()

    # approximate Q function using Neural Network
    def build_model(self):
        input_shape = self.state_size
        model = Sequential()
        # Write your code here: Add layers to your neural nets       
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model



    def get_action(self, state):
        action_indices, action_list = env.requests(state)
        if np.random.rand() <= self.epsilon:
        #if (0):
            self.explore += 1
            action_list_index = random.randrange(len(action_indices))
            action_space_index = action_indices[action_list_index]
        else:
            self.exploit += 1
            state = np.array(env.state_encod_arch1(state)).reshape(1, 36)
            q_value = self.model.predict(state)
            q_value =[q_value[0][i] for i in action_indices]
            #print("EXPLOIT")
            #print(q_value)
            #print(q_value[0])
            action_list_index = np.argmax(q_value)
            action_space_index = action_indices[action_list_index]
            #print(q_value[action_index])
        return action_space_index, action_list[action_list_index]
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment

    def append_sample(self, state, action_index, reward, next_state, done):
        self.memory.append((state, action_index, reward, next_state, done))
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
    
    
    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        #if (self.hit == True):
            #self.hit = False
            #q_value = self.model.predict(self.track_state)
        #print(q_value.shape)
        #print(q_value)
        #q_value =[q_value[0][i] for i in action_indices]
            #print('NEXT ITERATION AFTER HIT')
            #print(q_value)
        if len(self.memory) > self.batch_size:
            self.num_train += 1
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            # initialise two matrices - update_input and update_output
            update_input = np.zeros((self.batch_size, self.state_size))
            update_output = np.zeros((self.batch_size, self.state_size))
            actions, rewards, done = [], [], []

            # populate update_input and update_output and the lists rewards, actions, done
            for i in range(self.batch_size):
                state, action, reward, next_state, done_boolean = mini_batch[i]
                #print(state)
                #if (i == 0):
                    #print(type(update_input[i]))
                    #print(update_input[i].shape)
                    #print(type(state))
                update_input[i] = env.state_encod_arch1(state)
                if ((state_space.index(state) == 0) and (action == 2)):
                    self.num_train_track += 1
                    self.hit = True
                    self.hit_index = i
                    #print("TRAIN")
                    #print(state)
                    #print(update_input[i])
                    target = self.model.predict(update_input)
                    #print('TARGET')
                    #print(target[0])
                    #print(target[i], reward)
                    # target for q-network
                    target_qval = self.model.predict(update_output)
                    #print('TARGET_qval')
                    #print(target_qval[i])            
                actions.append(action)
                rewards.append(reward)
                update_output[i] = env.state_encod_arch1(next_state)
                done.append(done_boolean)

            # predict the target q-values from states s
            target = self.model.predict(update_input)
            # target for q-network
            target_qval = self.model.predict(update_output)


            # update the target values
            for i in range(self.batch_size):
                if done[i]:
                    target[i][actions[i]] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])
                    
            #if (self.hit == True):
                #print('AFTER TARGET CALC')
                #print(target[self.hit_index])
            # model fit
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
            
    def save_tracking_states(self):
        self.num_track += 1
            #print(state.shape)
            #q_value = self.model.predict(state_input[0])
        #print("TRACK")
        #print(self.track_state)
        q_value = self.model.predict(self.track_state)
            #print(q_value.shape)
            #print(q_value)
        #q_value =[q_value[0][i] for i in action_indices]
        self.states_tracked.append(q_value[0][2])

    def save(self, name):
        self.model.save(name)

In [14]:
Episodes = 1

### DQN block

for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    

    #Call the DQN agent
    
    
    while !terminal_state:
        
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        # 2. Evaluate your reward and next state
        # 3. Append the experience to the memory
        # 4. Train the model by calling function agent.train_model
        # 5. Keep a track of rewards, Q-values, loss
        

In [15]:
state_size = 36
action_size = 21
episode_time = 24*30
#n_episodes = 1
n_episodes = 3000
m = 5
t = 24
d = 7
num_track = 0
num_train = 0
num_track_train = 0
env = CabDriver()
agent = DQNAgent(action_size=action_size, state_size=state_size)


# to store rewards in each episode
rewards_per_episode, episodes = [], []

In [ ]:
start_time = time.time()
score_tracked = []
num_hits = 0
num_track = 0
num_train = 0
num_track_train = 0
for episode in range(n_episodes):

    done = False
    score = 0

    # reset at the start of each episode
    env = CabDriver()
    action_space, state_space, state = env.reset()
    state_size = m+t+d
    action_size = len(action_space)
    #agent = DQNAgent(state_size, action_size)
    #print(state)
    #print(state_space)
    total_time = 0
    step_num = 0
    while not done:
        step_num = step_num + 1
        #print(step_num)
        # get action for the current state and take a step in the environment
        action_index, action = agent.get_action(state)
        if ((state_space.index(state) == 0) and (action_index == 2)):
            num_hits = num_hits + 1
        #print(state, action)
        reward, next_state, step_time = env.step(state, action, Time_matrix)
        #print(state, next_state, reward)
        total_time += step_time
        if (total_time > episode_time):
            done = True
            #reward = 0
        # save the sample <s, a, r, s', done> to the replay memory
        agent.append_sample(state, action_index, reward, next_state, done)

        # train after each step
        agent.train_model()

        # add reward to the total score of this episode
        score += reward
        state = next_state


    # store total reward obtained in this episode
    rewards_per_episode.append(score)
    episodes.append(episode)

    # epsilon decay
    agent.epsilon = (1 - 0.00001) * np.exp(agent.epsilon_decay * episode)
    #agent.epsilon = agent.epsilon * 0.999 # for 2k
    #(1 - 0.0001) * np.exp(-0.003*i)
    #if agent.epsilon > agent.epsilon_min:
    #    agent.epsilon *= agent.epsilon_decay

    # every episode:
    if (episode % 1 == 0):
        print("episode {0}, reward {1}, memory_length {2}, epsilon {3} total_time {4} step_num {5}".format(episode,
                                                                         score,
                                                                         len(agent.memory),
                                                                         agent.epsilon, total_time, step_num))
        agent.save_tracking_states()
        score_tracked.append(score) 

elapsed_time = time.time() - start_time
#save_obj(States_track,'States_tracked')   
print(elapsed_time)
    # every few episodes:
        # store q-values of some prespecified state-action pairs
        # q_dict = agent.store_q_values()

        # save model weights
        #agent.save_model_weights(name="model_weights.h5")


episode 0, reward 129.0, memory_length 137, epsilon 0.99999 total_time 723.0 step_num 137
episode 1, reward -185.0, memory_length 277, epsilon 0.9992902519403939 total_time 730.0 step_num 140
episode 2, reward -49.0, memory_length 428, epsilon 0.9985909935330313 total_time 728.0 step_num 151
episode 3, reward -331.0, memory_length 564, epsilon 0.9978922244352755 total_time 725.0 step_num 136
episode 4, reward -86.0, memory_length 700, epsilon 0.9971939443047295 total_time 721.0 step_num 136
episode 5, reward 110.0, memory_length 847, epsilon 0.9964961527992363 total_time 725.0 step_num 147
episode 6, reward -313.0, memory_length 978, epsilon 0.9957988495768779 total_time 725.0 step_num 131
episode 7, reward -320.0, memory_length 1120, epsilon 0.9951020342959757 total_time 721.0 step_num 142
episode 8, reward -286.0, memory_length 1254, epsilon 0.9944057066150902 total_time 725.0 step_num 134
episode 9, reward -172.0, memory_length 1389, epsilon 0.9937098661930208 total_time 722.0 step_

episode 80, reward -229.0, memory_length 2000, epsilon 0.9455296804990374 total_time 728.0 step_num 133
episode 81, reward -54.0, memory_length 2000, epsilon 0.9448680413234165 total_time 729.0 step_num 114
episode 82, reward -186.0, memory_length 2000, epsilon 0.9442068651331547 total_time 723.0 step_num 127
episode 83, reward -365.0, memory_length 2000, epsilon 0.9435461516042758 total_time 721.0 step_num 140
episode 84, reward -118.0, memory_length 2000, epsilon 0.94288590041303 total_time 722.0 step_num 145
episode 85, reward -99.0, memory_length 2000, epsilon 0.9422261112358943 total_time 738.0 step_num 170
episode 86, reward 238.0, memory_length 2000, epsilon 0.9415667837495718 total_time 721.0 step_num 121
episode 87, reward -231.0, memory_length 2000, epsilon 0.9409079176309924 total_time 732.0 step_num 151
episode 88, reward -23.0, memory_length 2000, epsilon 0.9402495125573114 total_time 721.0 step_num 136
episode 89, reward -345.0, memory_length 2000, epsilon 0.9395915682059

episode 159, reward -484.0, memory_length 2000, epsilon 0.8946613608694933 total_time 725.0 step_num 136
episode 160, reward 94.0, memory_length 2000, epsilon 0.8940353170577823 total_time 730.0 step_num 148
episode 161, reward -64.0, memory_length 2000, epsilon 0.8934097113233944 total_time 722.0 step_num 129
episode 162, reward 97.0, memory_length 2000, epsilon 0.8927845433597831 total_time 724.0 step_num 150
episode 163, reward 82.0, memory_length 2000, epsilon 0.8921598128606157 total_time 736.0 step_num 130
episode 164, reward -93.0, memory_length 2000, epsilon 0.8915355195197746 total_time 726.0 step_num 126
episode 165, reward -239.0, memory_length 2000, epsilon 0.8909116630313557 total_time 721.0 step_num 122
episode 166, reward -108.0, memory_length 2000, epsilon 0.8902882430896697 total_time 729.0 step_num 135
episode 167, reward 197.0, memory_length 2000, epsilon 0.8896652593892407 total_time 722.0 step_num 121
episode 168, reward 221.0, memory_length 2000, epsilon 0.8890427

episode 238, reward 121.0, memory_length 2000, epsilon 0.8465296935051303 total_time 721.0 step_num 136
episode 239, reward 131.0, memory_length 2000, epsilon 0.8459373300710668 total_time 728.0 step_num 119
episode 240, reward -142.0, memory_length 2000, epsilon 0.845345381146312 total_time 734.0 step_num 132
episode 241, reward 17.0, memory_length 2000, epsilon 0.8447538464408108 total_time 731.0 step_num 133
episode 242, reward -13.0, memory_length 2000, epsilon 0.8441627256647113 total_time 737.0 step_num 131
episode 243, reward 173.0, memory_length 2000, epsilon 0.8435720185283643 total_time 725.0 step_num 122
episode 244, reward -266.0, memory_length 2000, epsilon 0.8429817247423231 total_time 721.0 step_num 131
episode 245, reward -415.0, memory_length 2000, epsilon 0.842391844017344 total_time 722.0 step_num 144
episode 246, reward -312.0, memory_length 2000, epsilon 0.8418023760643853 total_time 723.0 step_num 130
episode 247, reward -23.0, memory_length 2000, epsilon 0.841213

episode 317, reward 166.0, memory_length 2000, epsilon 0.8009874499212043 total_time 730.0 step_num 146
episode 318, reward -275.0, memory_length 2000, epsilon 0.8004269549024029 total_time 721.0 step_num 125
episode 319, reward -122.0, memory_length 2000, epsilon 0.7998668520928254 total_time 721.0 step_num 108
episode 320, reward -356.0, memory_length 2000, epsilon 0.7993071412180214 total_time 721.0 step_num 119
episode 321, reward -85.0, memory_length 2000, epsilon 0.7987478220037326 total_time 728.0 step_num 133
episode 322, reward -3.0, memory_length 2000, epsilon 0.7981888941758928 total_time 726.0 step_num 123
episode 323, reward 74.0, memory_length 2000, epsilon 0.7976303574606269 total_time 725.0 step_num 124
episode 324, reward -233.0, memory_length 2000, epsilon 0.7970722115842521 total_time 727.0 step_num 127
episode 325, reward 3.0, memory_length 2000, epsilon 0.7965144562732769 total_time 723.0 step_num 135
episode 326, reward -177.0, memory_length 2000, epsilon 0.795957

In [ ]:
print(agent.num_train, agent.num_track, agent.num_train_track, num_hits)

In [ ]:
print(agent.explore, agent.exploit)

In [ ]:
Time_matrix[0][4][0][0]

In [ ]:
Time_matrix[4][2][1][0]

In [ ]:
Time_matrix[2][0][2][0]

In [ ]:
Time_matrix[0][2][4][0]

In [ ]:
Time_matrix[2][1][6][0]

In [ ]:
Time_matrix[1][0][16][0]

In [ ]:
Time_matrix[0][3][20][0]

In [ ]:
Time_matrix[3][2][20][0]

In [ ]:
Time_matrix[2][3][23][0]

In [ ]:
env.state_get_loc(state)

### Tracking Convergence

In [ ]:
agent.states_tracked

In [ ]:
plt.figure(0, figsize=(16,7))
#plt.subplot(221)
plt.title('state [2,4,6]  action index 2')
xaxis = np.asarray(range(0, len(agent.states_tracked)))
plt.plot(xaxis,np.asarray(agent.states_tracked))
plt.show()

In [ ]:
plt.figure(0, figsize=(16,7))
#plt.subplot(221)
plt.title('state [2,4,6]  action index 2')
xaxis = np.asarray(range(0, len(score_tracked)))
plt.plot(xaxis,np.asarray(score_tracked))
plt.show()

In [ ]:
num_hits

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()

In [ ]:
time = np.arange(0,3000)
epsilon = []
for i in range(0,3000):
    epsilon.append(0 + (1 - 0.00001) * np.exp(-0.0007*i))
    z = np.random.random()

In [ ]:
time = np.arange(0,3000)
epsilon = []
epsilon_c = 1
for i in range(0,3000):
    epsilon.append(epsilon_c)
    epsilon_c = epsilon_c * 0.999

In [ ]:
plt.plot(time, epsilon)
plt.show()

In [ ]:
help(random.randrange)